# Preparation

<b>Load Libraries</b>

In [1]:
# udf and udc
import os, sys
cwd = os.getcwd()
os.chdir('../')
path_to_src = os.getcwd()
os.chdir(cwd)
if path_to_src not in sys.path:
    sys.path.append(path_to_src)
from src.notebook.support import *
from src.mlflow.support import *

# others
from warnings import simplefilter

<b>Load Dataset</b>

In [2]:
# materials
materials = prepare_data_to_train(path='../cache/data/cleaned.csv')

<b>MLflow</b>

In [3]:
# local server
mlflow.set_tracking_uri('http://127.0.0.1:5000/')
experiment_name = 'Feature engineering'
try:
    mlflow.create_experiment(name=experiment_name, artifact_location='../cache/mlflow/runs/')
except:
    mlflow.set_experiment(experiment_name=experiment_name)

# 1. LR && XGB

In [22]:
# LR as base model
## dowanlodad artifacts
run_id = mlflow.search_runs(
    filter_string="tags.base_model = 'LR' and tags.pre_pro = 'si(mean)_std_ohe_smoteenn'", 
    order_by=["metrics.val_fbeta DESC"]
)['run_id'].values[0]
try:
    path_to_lr = mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{run_id}/model/artifacts")
except:
    print("Check the above directory again!!")
##
feature_selector_lr = f"{path_to_lr}/feature_selector.pkl"
with open(feature_selector_lr, 'rb') as f:
    feature_selector_lr = cloudpickle.load(f)
##
lr = f"{path_to_lr}/model.pkl"
with open(lr, 'rb') as f:
    lr = cloudpickle.load(f)

lr_as_base = FSBaseClassifier(feature_selector_lr, lr)

In [29]:
# XGB as base model
## download artifacts
run_id = mlflow.search_runs(
    filter_string="tags.base_model = 'XGB' and tags.pre_pro = 'si(mean)_std_ohe_smoteenn'", 
    order_by=["metrics.val_fbeta DESC"]
)['run_id'].values[0]
try:
    path_to_xgb = mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{run_id}/model/artifacts")
except:
    print('Check the above directory again')
## 
feature_selector_xgb = f"{path_to_xgb}/feature_selector.pkl"
with open(feature_selector_xgb, 'rb') as f:
    feature_selector_xgb = cloudpickle.load(f)
##
xgb = f"{path_to_xgb}/model.pkl" 
with open(xgb, 'rb') as f:
    xgb = cloudpickle.load(f)

xgb_as_base = FSBaseClassifier(feature_selector_xgb, xgb)